In [1]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#filepath_met = "C:/Users/izzfa/Documents/Data Science/Year 2/Y2Q4/DC2/Data/Metropolitan-street.parquet"
#filepath_out = "C:/Users/izzfa/Documents/Data Science/Year 2/Y2Q4/DC2/Data/Metropolitan-outcomes.parquet"
#filepath_stop = "C:/Users/izzfa/Documents/Data Science/Year 2/Y2Q4/DC2/Data/Metropolitan-stopandsearch.parquet"


filepath_met = "C:/Users/Stan/OneDrive - TU Eindhoven/Tue Dingen/DataChallenge2/DataParquet/Metropolitan-street.parquet"

# read the Parquet file into a pandas dataframe
df = pq.read_pandas(filepath_met).to_pandas()


barnet_df = df[df['LSOA name'].str.contains('Barnet')==True]
barnetBur_df = barnet_df[barnet_df['Crime type'].str.contains('Burglary')==True]
barnetBur_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
2508,48d7fae1e76d275cfecabf17efbf6fecf769966089dcae...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.205163,51.655067,On or near Salisbury Road,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN
2509,d1d37e3e233d650abaf3eae1d6585e33f706cebe857ffc...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.203868,51.653824,On or near Union Street,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN
2532,9b09776a838d0d532429a57b320021da8bcffac6a2aa53...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.210051,51.657237,On or near Wentworth Road,E01000249,Barnet 001B,Burglary,Investigation complete; no suspect identified,NaN
2666,5c63da75bb8aa5fe30c925d6a4e58c3608bdd6fe68f33a...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.181376,51.648343,On or near Station Road,E01000251,Barnet 002D,Burglary,Investigation complete; no suspect identified,NaN
2667,6906ae484ce7bec7133bfdfa918cdcda566e04d71c465f...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.175660,51.651015,On or near Lytton Road,E01000251,Barnet 002D,Burglary,Investigation complete; no suspect identified,NaN


In [9]:
wards_csv = pd.read_csv("C:/Users/Stan/OneDrive - TU Eindhoven/Tue Dingen/DataChallenge2/DataParquet/LSOAtoWard.csv")
wards_relevant = wards_csv[['LSOA01NM', 'WD11NM']]
wards_relevant

,LSOA01NM,WD11NM
0,Barnet 020C,West Finchley
1,Barnet 019C,West Finchley
2,Barnet 020D,West Finchley
3,Barnet 020E,West Finchley
4,Barnet 036B,West Hendon
...,...,...
34373,Newport 001B,Caerleon
34374,Cardiff 019B,Cyncoed
34375,Newport 001C,Caerleon
34376,Cardiff 019C,Cyncoed


In [11]:
merged_df = pd.merge(left=barnetBur_df, right=wards_relevant, left_on='LSOA name', right_on='LSOA01NM')
merged_df.head()


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,LSOA01NM,WD11NM
0,48d7fae1e76d275cfecabf17efbf6fecf769966089dcae...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.205163,51.655067,On or near Salisbury Road,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet
1,d1d37e3e233d650abaf3eae1d6585e33f706cebe857ffc...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.203868,51.653824,On or near Union Street,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet
2,6b13c905e40eccf73feec600e87a6c70e0667b10980e95...,2020-07,Metropolitan Police Service,Metropolitan Police Service,-0.207839,51.654289,On or near The Avenue,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet
3,9869e99b7803ce398498332b862347626619671258ecdb...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.205204,51.656578,On or near Strafford Road,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet
4,83c73a8dfb5749e2f3922ff87ae87e3f0422e3cd8c88fa...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.203575,51.655735,On or near Shopping Area,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet


In [14]:
#Only 21 wards(should be 24) we have to check if this is different for data from a while ago
len(merged_df['WD11NM'].unique())

21

In [24]:
#Sklearn imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [25]:
#Labels and features of the data

y = merged_df['WD11NM']
X = merged_df['WD11NM']
#X = merged_df[['2Months' '2Months_lagg', '4Months', '4Months_lagg','6Months', 
#               '2Months_neighbor' '2Months_lagg_neighbor', '4Months_neighbor', 
#               '4Months_lagg_neighbor','6Months_neighbor']]

In [26]:
#Train/test split


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,
                                               random_state=42, shuffle=True) 

In [29]:
#Random forest classifier

rfc = RandomForestClassifier(random_state = 42)

rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
 
#Obtain accuracy

print("Test Score:",rfc.score(X_test,y_test))

ValueError: could not convert string to float: 'West Finchley'